In [ ]:
from flask import Flask, jsonify, send_file, request, session
import mariadb
import re
from collections import Counter
from wordcloud import WordCloud
import matplotlib.pyplot as plt
from flask_cors import CORS
from konlpy.tag import Okt

app = Flask(__name__)
app.secret_key = 'qwer1234'  #세션키
CORS(app)

# MariaDB 연결 설정
def get_db_connection():
    conn = mariadb.connect(
        user="root",
        password="1234",
        host="localhost",
        port=3306,
        database="ansan"
    )
    return conn

# 제외 리스트
korean_stopwords = set([
    '있다', '하다', '되다', '수', '을', '에', '의', '가', '이', '으로', '도', '를', '는', '다', '그', '그리고', '에서',
    '지음', '있는', '통해', '위해', '대한', '가장', '모든', '한다', '라는', '여러', '같은', '많은', '대해'
])

# 형태소분석기
okt = Okt()

# 텍스트 전처리 함수
def preprocess_text(text):
    text = re.sub(r'\W', ' ', text)  # 특수 문자 제거
    words = okt.nouns(text)  # 명사 추출
    words = [word for word in words if word not in korean_stopwords and len(word) > 1]  # 1글자 제외
    return words

# 검색어 저장 함수
def save_search_keyword(user_id, gender, keyword):
    conn = get_db_connection()
    cursor = conn.cursor()
    cursor.execute("INSERT INTO search_keywords (user_id, gender, keyword) VALUES (?, ?, ?)", (user_id, gender, keyword))
    conn.commit()
    conn.close()


dummy_data = {
    'MALE': ['데이터베이스', '프로그래밍', '알고리즘', '네트워크', '리눅스', '보안', '서버', '웹'],
    'FEMALE': ['디자인', '마케팅', '글쓰기', '브랜딩', '포토샵', '일러스트', '사진', '편집', '메타인지', '자료', '심리학', '소설',
              '데이터','클라우드', '네트워크', '파이썬', '알고리즘', '정보', '정보처리', '정보처리기사', '포토샵', '포토샵', '포토샵',
              '그래픽', '그래픽', '3D', '3D', '2D', '음악', '네트워크', '수능', '메타버스', '메타버스', '웹', '보안']
}

# 키워드 클라우드를 생성하는 함수
def create_wordcloud_for_gender(gender):
    conn = get_db_connection()
    cursor = conn.cursor()
    cursor.execute("SELECT keyword FROM search_keywords WHERE gender = ?", (gender,))
    keywords = [row[0] for row in cursor.fetchall()]
    conn.close()

    print(f"Keywords for {gender}: {keywords}")
    
    words = []
    for keyword in keywords:
        words.extend(preprocess_text(keyword))
    
    if gender in dummy_data:
        words.extend(dummy_data[gender])
    
    word_freq = Counter(words)
    print(f"Word frequencies for {gender}: {word_freq}")
    
    # 폰트 경로
    font_path = "D:\\python\\NanumGothic.ttf"

    wordcloud = WordCloud(width=800, height=400, background_color='white', font_path=font_path).generate_from_frequencies(word_freq)
    file_path = f"D:\\python\\wordcloud_{gender}.png"
    wordcloud.to_file(file_path)
    return file_path

@app.route('/api/wordcloud/<gender>', methods=['GET'])
def get_wordcloud_for_gender(gender):
    try:
        file_path = create_wordcloud_for_gender(gender)
        return send_file(file_path, mimetype='image/png')
    except FileNotFoundError as e:
        print(f"File not found: {e}")
        return jsonify({"error": "File not found"}), 404

@app.route('/api/save_keyword', methods=['POST'])
def save_keyword():
    data = request.json
    user_id = data.get("user_id")
    gender = data.get("gender")
    
    keyword = data.get('keyword')
    print(f"Saving keyword: {keyword} for user_id: {user_id} and gender: {gender}")
    if user_id and gender and keyword:
        save_search_keyword(user_id, gender, keyword)
        return jsonify({"status": "success"}), 200
    else:
        return jsonify({"error": "Invalid data"}), 400

@app.route('/api/login', methods=['POST'])
def login():
    data = request.json
    login_id = data.get('loginid')
    password = data.get('password')

    conn = get_db_connection()
    cursor = conn.cursor()
    cursor.execute("SELECT id, gender FROM users WHERE loginid = ? AND password = ?", (login_id, password))
    user = cursor.fetchone()
    conn.close()

    if user:
        session['user_id'] = user[0]
        session['gender'] = user[1]
        print(f"User logged in: {session['user_id']}, Gender: {session['gender']}") 
        return jsonify({"status": "success"}), 200
    else:
        return jsonify({"error": "Invalid credentials"}), 401

if __name__ == '__main__':
    app.run(port=5001)


 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:5001
Press CTRL+C to quit


Keywords for FEMALE: []
Word frequencies for FEMALE: Counter({'포토샵': 4, '네트워크': 2, '그래픽': 2, '3D': 2, '메타버스': 2, '디자인': 1, '마케팅': 1, '글쓰기': 1, '브랜딩': 1, '일러스트': 1, '사진': 1, '편집': 1, '메타인지': 1, '자료': 1, '심리학': 1, '소설': 1, '데이터': 1, '클라우드': 1, '파이썬': 1, '알고리즘': 1, '정보': 1, '정보처리': 1, '정보처리기사': 1, '2D': 1, '음악': 1, '수능': 1, '웹': 1, '보안': 1})
Keywords for MALE: []
Word frequencies for MALE: Counter({'데이터베이스': 1, '프로그래밍': 1, '알고리즘': 1, '네트워크': 1, '리눅스': 1, '보안': 1, '서버': 1, '웹': 1})


127.0.0.1 - - [29/Jul/2024 14:10:50] "GET /api/wordcloud/MALE HTTP/1.1" 200 -
127.0.0.1 - - [29/Jul/2024 14:10:50] "GET /api/wordcloud/FEMALE HTTP/1.1" 200 -
127.0.0.1 - - [29/Jul/2024 14:10:52] "OPTIONS /api/save_keyword HTTP/1.1" 200 -
127.0.0.1 - - [29/Jul/2024 14:10:52] "POST /api/save_keyword HTTP/1.1" 200 -


Saving keyword: 안산중앙도서관 for user_id: 3 and gender: FEMALE
Keywords for MALE: []
Word frequencies for MALE: Counter({'데이터베이스': 1, '프로그래밍': 1, '알고리즘': 1, '네트워크': 1, '리눅스': 1, '보안': 1, '서버': 1, '웹': 1})
Keywords for FEMALE: ['안산중앙도서관']


...

127.0.0.1 - - [29/Jul/2024 14:12:26] "GET /api/wordcloud/MALE HTTP/1.1" 200 -
127.0.0.1 - - [29/Jul/2024 14:12:26] "GET /api/wordcloud/FEMALE HTTP/1.1" 200 -


Keywords for MALE: []
Word frequencies for MALE: Counter({'데이터베이스': 1, '프로그래밍': 1, '알고리즘': 1, '네트워크': 1, '리눅스': 1, '보안': 1, '서버': 1, '웹': 1})
Keywords for FEMALE: ['안산중앙도서관', '안산중앙도서관']
Word frequencies for FEMALE: Counter({'포토샵': 4, '안산': 2, '중앙': 2, '도서관': 2, '네트워크': 2, '그래픽': 2, '3D': 2, '메타버스': 2, '디자인': 1, '마케팅': 1, '글쓰기': 1, '브랜딩': 1, '일러스트': 1, '사진': 1, '편집': 1, '메타인지': 1, '자료': 1, '심리학': 1, '소설': 1, '데이터': 1, '클라우드': 1, '파이썬': 1, '알고리즘': 1, '정보': 1, '정보처리': 1, '정보처리기사': 1, '2D': 1, '음악': 1, '수능': 1, '웹': 1, '보안': 1})


127.0.0.1 - - [29/Jul/2024 14:13:36] "GET /api/wordcloud/MALE HTTP/1.1" 200 -
127.0.0.1 - - [29/Jul/2024 14:13:36] "GET /api/wordcloud/FEMALE HTTP/1.1" 200 -
127.0.0.1 - - [29/Jul/2024 14:13:38] "OPTIONS /api/save_keyword HTTP/1.1" 200 -
127.0.0.1 - - [29/Jul/2024 14:13:38] "POST /api/save_keyword HTTP/1.1" 200 -


Saving keyword: 안산중앙도서관 for user_id: 4 and gender: MALE
Keywords for MALE: ['안산중앙도서관']
Keywords for FEMALE: ['안산중앙도서관', '안산중앙도서관']
Word frequencies for MALE: Counter({'안산': 1, '중앙': 1, '도서관': 1, '데이터베이스': 1, '프로그래밍': 1, '알고리즘': 1, '네트워크': 1, '리눅스': 1, '보안': 1, '서버': 1, '웹': 1})
Word frequencies for FEMALE: Counter({'포토샵': 4, '안산': 2, '중앙': 2, '도서관': 2, '네트워크': 2, '그래픽': 2, '3D': 2, '메타버스': 2, '디자인': 1, '마케팅': 1, '글쓰기': 1, '브랜딩': 1, '일러스트': 1, '사진': 1, '편집': 1, '메타인지': 1, '자료': 1, '심리학': 1, '소설': 1, '데이터': 1, '클라우드': 1, '파이썬': 1, '알고리즘': 1, '정보': 1, '정보처리': 1, '정보처리기사': 1, '2D': 1, '음악': 1, '수능': 1, '웹': 1, '보안': 1})


127.0.0.1 - - [29/Jul/2024 14:13:41] "GET /api/wordcloud/MALE HTTP/1.1" 200 -
127.0.0.1 - - [29/Jul/2024 14:13:41] "GET /api/wordcloud/FEMALE HTTP/1.1" 200 -


In [ ]:
from flask import Flask, jsonify, send_file, request
import mariadb
import re
from collections import Counter
from wordcloud import WordCloud
import matplotlib.pyplot as plt
from flask_cors import CORS
from konlpy.tag import Okt

app = Flask(__name__)
CORS(app)

# MariaDB
def get_db_connection():
    conn = mariadb.connect(
        user="root",
        password="1234",
        host="localhost",
        port=3306,
        database="ansan"
    )
    return conn

korean_stopwords = set([
    '있다', '하다', '되다', '수', '을', '에', '의', '가', '이', '으로', '도', '를', '는', '다', '그', '그리고', '에서',
    '지음', '있는', '통해', '위해', '대한', '가장', '모든', '한다', '라는', '여러', '같은', '많은', '대해'
])

# 형태소 분석기
okt = Okt()

# 텍스트 필터
def preprocess_text(text):
    text = re.sub(r'\W', ' ', text)  # 특수 문자 제거
    words = okt.nouns(text)  # 명사 추출
    words = [word for word in words if word not in korean_stopwords and len(word) > 1]  # 한글 불용어 및 1글자 이하 단어 제거
    return words

# 검색어 저장 함수 (lib_user 테이블에 저장)
def save_search_keyword(user_id, gender, keyword):
    conn = get_db_connection()
    cursor = conn.cursor()
    cursor.execute("INSERT INTO search_keywords (user_id, gender, keyword) VALUES (?, ?, ?)", (user_id, gender, keyword))
    conn.commit()
    conn.close()


dummy_data = {
    'MALE': ['데이터베이스', '프로그래밍', '알고리즘', '네트워크', '리눅스', '보안', '서버', '웹'],
    'FEMALE': ['디자인', '마케팅', '글쓰기', '브랜딩', '포토샵', '일러스트', '사진', '편집', '메타인지', '자료', '심리학', '소설',
              '데이터','클라우드', '네트워크', '파이썬', '알고리즘', '정보', '정보처리', '정보처리기사', '포토샵', '포토샵', '포토샵',
              '그래픽', '그래픽', '3D', '3D', '2D', '음악', '네트워크', '수능', '메타버스', '메타버스', '웹', '보안']
}

# 키워드 클라우드를 생성하는 함수
def create_wordcloud_for_gender(gender):
    conn = get_db_connection()
    cursor = conn.cursor()
    cursor.execute("SELECT keyword FROM search_keywords WHERE gender = ?", (gender,))
    keywords = [row[0] for row in cursor.fetchall()]
    conn.close()
    
    words = []
    for keyword in keywords:
        words.extend(preprocess_text(keyword))
    
    if not words and gender in dummy_data:
        words.extend(dummy_data[gender])
    
    word_freq = Counter(words)
    
    # 폰트 경로
    font_path = "D:\\python\\NanumGothic.ttf"

    wordcloud = WordCloud(width=500, height=250, background_color='white', font_path=font_path).generate_from_frequencies(word_freq)
    file_path = f"D:\\python\\wordcloud_{gender}.png"
    wordcloud.to_file(file_path)
    return file_path

@app.route('/api/wordcloud/<gender>', methods=['GET'])
def get_wordcloud_for_gender(gender):
    try:
        file_path = create_wordcloud_for_gender(gender)
        return send_file(file_path, mimetype='image/png')
    except FileNotFoundError as e:
        print(f"File not found: {e}")
        return jsonify({"error": "File not found"}), 404

@app.route('/api/save_keyword', methods=['POST'])
def save_keyword():
    data = request.json
    user_id = data.get('user_id')
    gender = data.get('gender')
    keyword = data.get('keyword')
    if user_id and gender and keyword:
        save_search_keyword(user_id, gender, keyword)
        return jsonify({"status": "success"}), 200
    else:
        return jsonify({"error": "Invalid data"}), 400

if __name__ == '__main__':
    app.run(port=5001)
